In [1]:
import pandas as pd, numpy as np, warnings, os
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
warnings.filterwarnings('ignore')

In [3]:
pwd

'C:\\Users\\ABC\\3. Edureka\\20. Website deployment\\Algo 11 - Continuous variable predictor'

In [4]:
os.chdir('C:\\Users\\ABC\\6. Datasets')

In [5]:
df = pd.read_csv('superstore_data_Classifier.csv')
df.columns

Index(['Id', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'Complain', 'Response'],
      dtype='object')

In [6]:
df

,Id,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response
0,1826,1970,Graduation,Divorced,84835.0,0,0,6/16/2014,0,189,...,111,189,218,1,4,4,6,1,0,1
1,1,1961,Graduation,Single,57091.0,0,0,6/15/2014,0,464,...,7,0,37,1,7,3,7,5,0,1
2,10476,1958,Graduation,Married,67267.0,0,1,5/13/2014,0,134,...,15,2,30,1,3,2,5,2,0,0
3,1386,1967,Graduation,Together,32474.0,1,1,11/5/2014,0,10,...,0,0,0,1,1,0,2,7,0,0
4,5371,1989,Graduation,Single,21474.0,1,0,8/4/2014,0,6,...,11,0,34,2,3,1,2,7,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10142,1976,PhD,Divorced,66476.0,0,1,7/3/2013,99,372,...,47,48,78,2,5,2,11,4,0,0
2236,5263,1977,2n Cycle,Married,31056.0,1,0,1/22/2013,99,5,...,3,8,16,1,1,0,3,8,0,0
2237,22,1976,Graduation,Divorced,46310.0,1,0,3/12/2012,99,185,...,15,5,14,2,6,1,5,8,0,0
2238,528,1978,Graduation,Married,65819.0,0,0,11/29/2012,99,267,...,149,165,63,1,5,4,10,3,0,0


In [7]:
df.columns
df.drop(['Id', 'Year_Birth', 'Dt_Customer'], inplace=True, axis = 1)

In [8]:
df['Education'] = df['Education'].map({
    'Basic':0,
    '2n Cycle':1,
    'Graduation':2,
    'Master':3,
    'PhD':4   
})

In [9]:
#Checking the below proportions of 1vs0 for double vs single,  singles opt in for schemes a lot more.
#SO giving them a 2
df['Marital_Status'].value_counts()

df['Marital_Status'] = df['Marital_Status'].map({
    'Married':1,
    'Together':1,
    'Single':2,
    'Widow':2,
    'Divorced':2,
    'YOLO':2,
    'Alone':2,
    'Absurd':2
})

In [10]:
listy = []
for i in df['Marital_Status'].values:
    if i not in listy:
        listy.append(i)
listy

[2, 1]

In [11]:
df[df['Marital_Status']=='Divorced']['Response'].value_counts()


Series([], Name: Response, dtype: int64)

In [12]:
df[df['Marital_Status']=='Together']['Response'].value_counts()


Series([], Name: Response, dtype: int64)

In [13]:
df[df['Marital_Status']=='Single']['Response'].value_counts()


Series([], Name: Response, dtype: int64)

In [14]:
df[df['Marital_Status']=='Widow']['Response'].value_counts()


Series([], Name: Response, dtype: int64)

In [15]:
df[df['Marital_Status']=='Married']['Response'].value_counts()


Series([], Name: Response, dtype: int64)

In [16]:
df

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response
0,2,2,84835.0,0,0,0,189,104,379,111,189,218,1,4,4,6,1,0,1
1,2,2,57091.0,0,0,0,464,5,64,7,0,37,1,7,3,7,5,0,1
2,2,1,67267.0,0,1,0,134,11,59,15,2,30,1,3,2,5,2,0,0
3,2,1,32474.0,1,1,0,10,0,1,0,0,0,1,1,0,2,7,0,0
4,2,2,21474.0,1,0,0,6,16,24,11,0,34,2,3,1,2,7,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,4,2,66476.0,0,1,99,372,18,126,47,48,78,2,5,2,11,4,0,0
2236,1,1,31056.0,1,0,99,5,10,13,3,8,16,1,1,0,3,8,0,0
2237,2,2,46310.0,1,0,99,185,2,88,15,5,14,2,6,1,5,8,0,0
2238,2,1,65819.0,0,0,99,267,38,701,149,165,63,1,5,4,10,3,0,0


In [17]:
df['Income'].fillna(52247.251354, inplace=True)

In [18]:
#People who bought into the product v/s thse who didnt complain!
df[df['Response']==1]['Complain'].value_counts()

0    331
1      3
Name: Complain, dtype: int64

In [19]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [20]:
df.isnull().sum()

Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
Complain               0
Response               0
dtype: int64

In [21]:
#Logistic Regresion
for i in range(10):
    x_train_lr, x_test_lr, y_train_lr, y_test_lr = train_test_split(x,y,test_size=0.2, random_state=i)
    lg = LogisticRegression(max_iter=1000)
    model_lr = lg.fit(x_train_lr, y_train_lr)
    y_predict_lr = model_lr.predict(x_test_lr)
    acc_lr = accuracy_score(y_predict_lr, y_test_lr)
    cv = cross_val_score(model_lr, x, y, cv=5, scoring='accuracy')
    if acc_lr>0.59:
        print(i, acc_lr)
        print(cv)

0 0.8816964285714286
[0.74107143 0.86160714 0.86830357 0.86830357 0.85491071]
1 0.8482142857142857
[0.74107143 0.86160714 0.86830357 0.86830357 0.85491071]
2 0.859375
[0.74107143 0.86160714 0.86830357 0.86830357 0.85491071]
3 0.8571428571428571
[0.74107143 0.86160714 0.86830357 0.86830357 0.85491071]
4 0.8392857142857143
[0.74107143 0.86160714 0.86830357 0.86830357 0.85491071]
5 0.8348214285714286
[0.74107143 0.86160714 0.86830357 0.86830357 0.85491071]
6 0.8504464285714286
[0.74107143 0.86160714 0.86830357 0.86830357 0.85491071]
7 0.8772321428571429
[0.74107143 0.86160714 0.86830357 0.86830357 0.85491071]
8 0.8459821428571429
[0.74107143 0.86160714 0.86830357 0.86830357 0.85491071]
9 0.8660714285714286
[0.74107143 0.86160714 0.86830357 0.86830357 0.85491071]


In [22]:
#Decision Tree
for j in range(500):
    x_train_dt, x_test_dt, y_train_dt, y_test_dt = train_test_split(x,y,test_size=0.2, random_state=j)
    dt = DecisionTreeClassifier()
    model_dt = dt.fit(x_train_dt, y_train_dt)
    y_predict_dt = model_dt.predict(x_test_dt)
    acc_dt = accuracy_score(y_test_dt, y_predict_dt)
    cv = cross_val_score(model_dt, x, y, cv=5, scoring='accuracy')
    if acc_dt > 0.85:
        print(j, acc_dt)
        print(cv)

6 0.8638392857142857
[0.16964286 0.05803571 0.09151786 0.10491071 0.85044643]
28 0.8526785714285714
[0.16964286 0.05133929 0.08482143 0.11160714 0.85044643]
36 0.859375
[0.16964286 0.05803571 0.08705357 0.12276786 0.85044643]
80 0.8526785714285714
[0.16964286 0.0625     0.08482143 0.10491071 0.85044643]
144 0.8549107142857143
[0.16741071 0.05133929 0.08705357 0.11160714 0.85044643]
151 0.8504464285714286
[0.16964286 0.06473214 0.08705357 0.10714286 0.85044643]
166 0.8571428571428571
[0.16964286 0.05580357 0.08482143 0.10267857 0.85044643]
194 0.859375
[0.16741071 0.06026786 0.07589286 0.10267857 0.85044643]
228 0.8571428571428571
[0.16964286 0.05803571 0.08482143 0.12053571 0.85044643]
235 0.8549107142857143
[0.16964286 0.0625     0.078125   0.11830357 0.85044643]
245 0.8616071428571429
[0.16964286 0.06026786 0.08258929 0.11607143 0.85044643]
252 0.8526785714285714
[0.16964286 0.05803571 0.08705357 0.12276786 0.85044643]
292 0.8616071428571429
[0.16964286 0.05580357 0.08705357 0.102678

In [23]:
#RandomForest
for k in range(10):
    x_train_rf, x_test_rf, y_train_rf,y_test_rf = train_test_split(x,y,test_size=0.2, random_state=k)
    rf = RandomForestClassifier(
    n_estimators=400,       # Keep as is
    max_depth=20,           # Limit depth to prevent overfitting
    min_samples_split=5,    # Increase a bit more
    min_samples_leaf=3,     # Prevent small splits
    max_features='sqrt',    # Common for classification
    bootstrap=True,         # Helps with generalization
    class_weight='balanced' # Adjusts for any class imbalance
)
    model_rf = rf.fit(x_train_rf, y_train_rf)
    cv = cross_val_score(model_rf, x, y, cv=5, scoring='accuracy')
    y_predict_rf = model_rf.predict(x_test_rf)
    acc_rf = accuracy_score(y_test_rf, y_predict_rf)
    if acc_rf>0.88:
        print(k, acc_rf)
        print(cv)

0 0.8883928571428571
[0.17410714 0.59375    0.80133929 0.83705357 0.85267857]
2 0.8928571428571429
[0.171875   0.64955357 0.80357143 0.828125   0.85267857]
4 0.8883928571428571
[0.17633929 0.64285714 0.80133929 0.83035714 0.85267857]
6 0.890625
[0.17410714 0.60714286 0.80133929 0.83258929 0.85267857]
7 0.8928571428571429
[0.16964286 0.62723214 0.80803571 0.83035714 0.85267857]
9 0.9040178571428571
[0.171875   0.61830357 0.796875   0.83482143 0.85267857]


In [24]:
rf.feature_importances_

array([2.13470564e-02, 3.12130709e-02, 9.66384991e-02, 7.37824086e-03,
       2.90088167e-02, 1.37152927e-01, 9.98322558e-02, 4.29906844e-02,
       1.05532186e-01, 4.76714576e-02, 5.08018213e-02, 9.56140428e-02,
       2.61938832e-02, 3.66645242e-02, 6.42518392e-02, 5.34468498e-02,
       5.42193227e-02, 4.25222624e-05])

In [25]:
params=parameters = {'max_depth' : (3,5,7,9,10,15,20,25)
              , 'criterion' : ('gini', 'entropy')
              , 'max_features' : ('auto', 'sqrt', 'log2')
              , 'min_samples_split' : (2,4,6)
             }

In [26]:
rs = RandomizedSearchCV(dt, param_distributions= params, cv = 5, verbose = True)

In [27]:
x_train_dt, x_test_dt, y_train_dt, y_test_dt = train_test_split(x,y,test_size=0.2, random_state=42)
dt = DecisionTreeClassifier()
model_dt = dt.fit(x_train_dt, y_train_dt)
y_predict_dt = model_dt.predict(x_test_dt)
acc_dt = accuracy_score(y_test_dt, y_predict_dt)
cv = cross_val_score(model_dt, x, y, cv=5, scoring='accuracy')
rs.fit(x_train_dt, y_train_dt)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(),
                   param_distributions={'criterion': ('gini', 'entropy'),
                                        'max_depth': (3, 5, 7, 9, 10, 15, 20,
                                                      25),
                                        'max_features': ('auto', 'sqrt',
                                                         'log2'),
                                        'min_samples_split': (2, 4, 6)},
                   verbose=True)

In [28]:
rs.best_estimator_


DecisionTreeClassifier(max_depth=3, max_features='sqrt', min_samples_split=6)

In [29]:
dt = DecisionTreeClassifier(criterion='entropy', max_depth=3, max_features='log2',
                       min_samples_split=6)
model_dt = dt.fit(x_train_dt, y_train_dt)
y_predict_dt = model_dt.predict(x_test_dt)
acc_dt = accuracy_score(y_test_dt, y_predict_dt)
cv = cross_val_score(model_dt, x, y, cv=5, scoring='accuracy')
acc_dt
print(cv, acc_dt)

[0.68973214 0.84821429 0.85044643 0.85044643 0.85267857] 0.8415178571428571


In [30]:
#KNN
import warnings
# Suppress FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
try: 
    for l in range(278, 283):
        x_train_knn, x_test_knn, y_train_knn, y_test_knn = train_test_split(x,y,test_size=0.2, 
                                                                            random_state=l)
        knn = KNeighborsClassifier()
        model_knn = knn.fit(x_train_knn, y_train_knn)
        y_predict_knn = model_knn.predict(x_test_knn)
        acc_knn = accuracy_score(y_predict_knn, y_test_knn)
        if acc_knn > 0.75:
            print(l, acc_knn)
        cv = cross_val_score(knn, x,y, cv=5, scoring='accuracy')
        print(cv)
except ValueError:
    print("Invalid")
    

278 0.8392857142857143
[0.82589286 0.80580357 0.83705357 0.84821429 0.83035714]
279 0.8325892857142857
[0.82589286 0.80580357 0.83705357 0.84821429 0.83035714]
280 0.8325892857142857
[0.82589286 0.80580357 0.83705357 0.84821429 0.83035714]
281 0.8370535714285714
[0.82589286 0.80580357 0.83705357 0.84821429 0.83035714]
282 0.8303571428571429
[0.82589286 0.80580357 0.83705357 0.84821429 0.83035714]


In [31]:
#Support Vector Machines
from sklearn.utils import shuffle
x,y = shuffle(x,y,random_state=42)
for m in range(10):
    x_train_svc, x_test_svc, y_train_svc, y_test_svc = train_test_split(x,y,test_size = 0.2,
                                                                        random_state = m)
    svc = SVC(C=0.1)
    model_svc = svc.fit(x_train_svc, y_train_svc)
    y_predict_svc = model_svc.predict(x_test_svc)
    acc_svc = accuracy_score(y_test_svc, y_predict_svc)
    cv = cross_val_score(svc, x,y,scoring='accuracy', cv=StratifiedKFold(n_splits=5))
    if acc_svc > 0.85:
        print(m, cv, acc_svc)

1 [0.85044643 0.85044643 0.85044643 0.85044643 0.85267857] 0.8683035714285714
2 [0.85044643 0.85044643 0.85044643 0.85044643 0.85267857] 0.859375
3 [0.85044643 0.85044643 0.85044643 0.85044643 0.85267857] 0.859375
4 [0.85044643 0.85044643 0.85044643 0.85044643 0.85267857] 0.8683035714285714
5 [0.85044643 0.85044643 0.85044643 0.85044643 0.85267857] 0.8660714285714286
7 [0.85044643 0.85044643 0.85044643 0.85044643 0.85267857] 0.859375


In [32]:
#GradientBoostingClassifier
x,y = shuffle(x,y, random_state=42)
for n in range(5):
    x_train_gb, x_test_gb, y_train_gb, y_test_gb = train_test_split(x,y,test_size = 0.2, 
                                                                    random_state = n)
    gb = GradientBoostingClassifier()
    model_gb = gb.fit(x_train_gb, y_train_gb)
    y_predict_gb = gb.predict(x_test_gb)
    acc_gb = accuracy_score(y_predict_gb, y_test_gb)
    cv = cross_val_score(gb, x,y,scoring='accuracy', cv=StratifiedKFold(n_splits=5))
    if acc_gb > 0.87:
        print(n, acc_gb)
        print(cv)

1 0.8794642857142857
[0.86160714 0.86830357 0.875      0.87053571 0.89285714]
2 0.8839285714285714
[0.86160714 0.86830357 0.875      0.87053571 0.89285714]


In [33]:
x_train_gb, x_test_gb, y_train_gb, y_test_gb = train_test_split(x,y,test_size = 0.2, 
                                                                random_state = 3)
gb = GradientBoostingClassifier()
model_gb = gb.fit(x_train_gb, y_train_gb)
y_predict_gb = gb.predict(x_test_gb)
acc_gb = accuracy_score(y_predict_gb, y_test_gb)
cv = cross_val_score(gb, x,y,scoring='accuracy', cv=StratifiedKFold(n_splits=5))
if acc_gb > 0.85:
    print(n, acc_gb)
    print(cv)

4 0.8638392857142857
[0.86160714 0.87053571 0.875      0.87053571 0.89285714]


In [34]:
import pickle
with open('savedmde.sav1', 'wb') as f1:
    pickle.dump(model_gb, f1)

In [35]:
with open('savedmde.sav1', 'rb') as f2:
    loaded_model = pickle.load(f2)

In [36]:
loaded_model.predict_proba([[2,2,84800,0,0,0,189,104,379,111,189,218,1,4,4,6,1,0]])[0][1]

0.7802037682533451

In [37]:
df.head(3)

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response
0,2,2,84835.0,0,0,0,189,104,379,111,189,218,1,4,4,6,1,0,1
1,2,2,57091.0,0,0,0,464,5,64,7,0,37,1,7,3,7,5,0,1
2,2,1,67267.0,0,1,0,134,11,59,15,2,30,1,3,2,5,2,0,0


In [38]:
from sklearn.feature_selection import RFE

In [39]:
rfe = RFE(model_gb, n_features_to_select=9)

In [40]:
fitter = rfe.fit(x,y)

In [41]:
#Teh features:
feat = []
[feat.append(i) for i,j in enumerate(fitter.support_) if j == True]
feat

[1, 2, 5, 6, 8, 11, 14, 15, 16]

In [42]:
#YAAAAASSSSS!!!!!!!!!!! 
df.iloc[:,[1, 2, 5, 6, 8, 11, 14, 15, 16]]

,Marital_Status,Income,Recency,MntWines,MntMeatProducts,MntGoldProds,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth
0,2,84835.0,0,189,379,218,4,6,1
1,2,57091.0,0,464,64,37,3,7,5
2,1,67267.0,0,134,59,30,2,5,2
3,1,32474.0,0,10,1,0,0,2,7
4,2,21474.0,0,6,24,34,1,2,7
...,...,...,...,...,...,...,...,...,...
2235,2,66476.0,99,372,126,78,2,11,4
2236,1,31056.0,99,5,13,16,0,3,8
2237,2,46310.0,99,185,88,14,1,5,8
2238,1,65819.0,99,267,701,63,4,10,3


In [43]:
"""from sklearn.feature_selection import RFE
from sklearn.svm import SVC

model = SVC(kernel='linear')
rfe = RFE(model, n_features_to_select=9)
fit = rfe.fit(x, y)

print(fit.support_)  # Features selected
print(fit.ranking_)  # Ranking of features"""

"from sklearn.feature_selection import RFE\nfrom sklearn.svm import SVC\n\nmodel = SVC(kernel='linear')\nrfe = RFE(model, n_features_to_select=9)\nfit = rfe.fit(x, y)\n\nprint(fit.support_)  # Features selected\nprint(fit.ranking_)  # Ranking of features"

In [44]:
x1 = df.iloc[:,[1, 2, 5, 6, 8, 11, 14, 15, 16]]
y = df.iloc[:,-1]

In [45]:
x_train_gb, x_test_gb, y_train_gb, y_test_gb = train_test_split(x,y,test_size = 0.2, 
                                                                random_state = 3)
gb = GradientBoostingClassifier()
model_gb = gb.fit(x_train_gb, y_train_gb)
y_predict_gb = gb.predict(x_test_gb)
acc_gb = accuracy_score(y_predict_gb, y_test_gb)
cv = cross_val_score(gb, x,y,scoring='accuracy', cv=StratifiedKFold(n_splits=5))
if acc_gb > 0.87:
    print(n, acc_gb)
    print(cv)